# Testing Playground for r5py

In [1]:
import r5py

In [2]:
import geopandas
import shapely

## Acquiring and Preprocessing Datasets

### Road Networks from OSM

In [3]:
import zipfile
import pandas as pd


In [4]:
with zipfile.ZipFile('/home/emily/thesis_BA/data/gtfs/2023_rnv_gtfs.zip') as gtfs:
        with gtfs.open('stops.txt') as stops_file:
            stops_df = pd.read_table(stops_file, sep=",")
stops_df

,stop_id,stop_name,stop_lat,stop_lon,location_type,platform_code
0,11601,Bad Dürkheim Bahnhof,49.461511,8.170402,0,1
1,13401,Bad Dürkheim Ost,49.463437,8.185054,0,1
2,13402,Bad Dürkheim Ost,49.463315,8.184245,0,2
3,51501,Dossenheim Bahnhof,49.448191,8.670171,0,1
4,51502,Dossenheim Bahnhof,49.448553,8.669533,0,2
...,...,...,...,...,...,...
2074,752112,Rheinallee Süd,49.473887,8.455357,0,B
2075,752212,Drehbrücke,49.470350,8.452832,0,A
2076,752611,Rheinallee Nord,49.476560,8.455411,0,A
2077,814711,MA Waldorfschule,49.449312,8.473036,0,A


In [5]:
stops_gdf = geopandas.GeoDataFrame(
    stops_df, geometry=geopandas.points_from_xy(stops_df.stop_lon, stops_df.stop_lat), crs="EPSG:4326"
)
stops_gdf


,stop_id,stop_name,stop_lat,stop_lon,location_type,platform_code,geometry
0,11601,Bad Dürkheim Bahnhof,49.461511,8.170402,0,1,POINT (8.17040 49.46151)
1,13401,Bad Dürkheim Ost,49.463437,8.185054,0,1,POINT (8.18505 49.46344)
2,13402,Bad Dürkheim Ost,49.463315,8.184245,0,2,POINT (8.18425 49.46332)
3,51501,Dossenheim Bahnhof,49.448191,8.670171,0,1,POINT (8.67017 49.44819)
4,51502,Dossenheim Bahnhof,49.448553,8.669533,0,2,POINT (8.66953 49.44855)
...,...,...,...,...,...,...,...
2074,752112,Rheinallee Süd,49.473887,8.455357,0,B,POINT (8.45536 49.47389)
2075,752212,Drehbrücke,49.470350,8.452832,0,A,POINT (8.45283 49.47035)
2076,752611,Rheinallee Nord,49.476560,8.455411,0,A,POINT (8.45541 49.47656)
2077,814711,MA Waldorfschule,49.449312,8.473036,0,A,POINT (8.47304 49.44931)


In [6]:
stops_gdf.explore()

In [7]:
geofabrik_available = geopandas.read_file('/home/emily/thesis_BA/data/geofabrik_downloadindex.json')

#finding smallest available set covering gtfs feed
matching_datasets = geofabrik_available.contains(stops_gdf.unary_union)
coverage = geofabrik_available[matching_datasets]['geometry'].area
smallest = coverage.idxmin()
preferred_set = geofabrik_available.iloc[smallest]['id']

/tmp/ipykernel_2390/1406673922.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coverage = geofabrik_available[matching_datasets]['geometry'].area


In [8]:
from pathlib import Path
#check if parent dataset already downloaded
parent_set = geofabrik_available.iloc[smallest]['parent']
osm_data = None
for path in Path('/home/emily/thesis_BA/data/osm_data').glob(f"{parent_set}*"):
    osm_data = path

In [9]:
import pyrosm
if osm_data is None:
    osm_data = pyrosm.get_data(dataset=preferred_set, directory='/home/emily/thesis_BA/data/osm_data')

In [10]:
stop_bounds = stops_gdf.total_bounds
left, bottom, right, top = stop_bounds
type(top)

numpy.float64

In [11]:
'''
import subprocess

#cropping pbf to bounding box with osmosis
subprocess.run(['osmosis', '--read-pbf', f'file={osm_data}', '--bounding-box', f'top={top}', f'left={left}', f'bottom={bottom}', f'right={right}', 'completeWays=yes', '--write-pbf', f'file=/home/emily/thesis_BA/data/osm_data/test.osm.pbf'])
'''

"\nimport subprocess\n\n#cropping pbf to bounding box with osmosis\nsubprocess.run(['osmosis', '--read-pbf', f'file={osm_data}', '--bounding-box', f'top={top}', f'left={left}', f'bottom={bottom}', f'right={right}', 'completeWays=yes', '--write-pbf', f'file=/home/emily/thesis_BA/data/osm_data/test.osm.pbf'])\n"

this takes sooooooooo long

In [12]:

transport_network_rnv = r5py.TransportNetwork(
    osm_pbf='/home/emily/thesis_BA/data/osm_data/test.osm.pbf',
    gtfs='/home/emily/thesis_BA/data/gtfs/2023_rnv_gtfs.zip',
)

need to extract some stuff from the pbf too to have destinations

In [13]:
osm_data_obj = pyrosm.pyrosm.OSM('/home/emily/thesis_BA/data/osm_data/test.osm.pbf')
osm_data_obj.get_boundaries()

,visible,boundary,id,timestamp,version,tags,osm_type,geometry,name,ref,website,admin_level,border_type,changeset
0,False,administrative,162319720,1555672285,3,"{""note"":""Gemarkungsgrenze Flomersheim""}",way,"MULTILINESTRING ((8.33122 49.52966, 8.33389 49...",NaN,NaN,NaN,NaN,NaN,NaN
1,False,administrative,162319721,1555672285,4,"{""note"":""Gemarkungsgrenze M\u00F6rsch""}",way,"MULTILINESTRING ((8.36367 49.54444, 8.36472 49...",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,administrative,639343,1695675901,300,"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,"MULTIPOLYGON (((8.60334 49.59385, 8.75477 49.4...",Regierungsbezirk Karlsruhe,None,None,5,None,0.0
3,NaN,administrative,8462063,1701029748,472,"{""ISO3166-2"":""DE-RP"",""TMC:cid_58:tabcd_1:Class...",relation,"MULTIPOLYGON (((8.42250 49.58372, 8.42303 49.5...",Rheinland-Pfalz,RLP,None,4,state,0.0
4,NaN,administrative,8524410,1695967346,38,"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,"POLYGON ((8.29988 49.47387, 8.29983 49.47397, ...",Ludwigshafen am Rhein,None,None,6,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,NaN,administrative,34943106648,1693845044,2,"{""description"":""Das Gebiet der Gemarkung L\u00...",relation,"POLYGON ((8.72528 49.54759, 8.72590 49.54779, ...",Löhrbach,None,None,9,None,0.0
241,NaN,administrative,34959371635,1693551353,1,"{""description"":""Das Gebiet des Ortsteils Reise...",relation,"POLYGON ((8.71938 49.58355, 8.71936 49.58339, ...",Reisen,None,None,9,None,0.0
242,NaN,administrative,34975636637,1693551802,1,"{""description"":""Das Gebiet des Ortsteils Niede...",relation,"POLYGON ((8.71074 49.58194, 8.71043 49.58190, ...",Nieder-Liebersbach,None,None,9,None,0.0
243,NaN,administrative,34991901864,1693552721,1,"{""description"":""Das Gebiet des Ortsbezirks Hor...",relation,"POLYGON ((8.71890 49.57077, 8.71956 49.57011, ...",Hornbach,None,None,9,None,0.0


In [14]:
admin_boundaries = osm_data_obj.get_boundaries()
counties = admin_boundaries.loc[admin_boundaries['admin_level'] == '6']
counties

,visible,boundary,id,timestamp,version,geometry,tags,osm_type,name,ref,website,admin_level,border_type,changeset
4,NaN,administrative,8524410,1695967346,38,"POLYGON ((8.29988 49.47387, 8.29983 49.47397, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Ludwigshafen am Rhein,None,None,6,None,0.0
5,NaN,administrative,8586762,1632578425,21,"POLYGON ((8.40363 49.30526, 8.40359 49.30541, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Speyer,None,None,6,None,0.0
6,NaN,administrative,8649216,1699197735,57,"MULTIPOLYGON (((8.15181 49.39231, 8.15294 49.3...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Landkreis Bad Dürkheim,None,None,6,county,0.0
7,NaN,administrative,8711789,1652953400,28,"POLYGON ((8.31071 49.50171, 8.31073 49.50210, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Frankenthal (Pfalz),None,None,6,None,0.0
8,NaN,administrative,8774366,1695228268,140,"MULTIPOLYGON (((8.60213 49.40930, 8.47028 49.3...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Rhein-Neckar-Kreis,None,None,6,None,0.0
9,NaN,administrative,8836976,1695967346,52,"POLYGON ((8.28099 49.58687, 8.28093 49.58408, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Rhein-Pfalz-Kreis,None,None,6,None,0.0
12,NaN,administrative,9024928,1662078812,55,"POLYGON ((8.41748 49.54873, 8.41752 49.54868, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Mannheim,None,None,6,None,0.0
13,NaN,administrative,9087652,1689220939,34,"MULTIPOLYGON (((8.14263 49.39446, 8.14514 49.3...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Neustadt an der Weinstraße,None,None,6,None,0.0
14,NaN,administrative,9150382,1693474719,120,"MULTIPOLYGON (((8.48176 49.57494, 8.49260 49.5...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Kreis Bergstraße,None,None,6,None,0.0
27,NaN,administrative,53278047,1695228268,32,"POLYGON ((8.57343 49.41576, 8.57329 49.41652, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0


In [15]:
school_filter = {"amenity": ["school"]}
schools = osm_data_obj.get_data_by_custom_criteria(custom_filter=school_filter)
schools_points = schools.copy()
schools_points['geometry']= schools.centroid
schools_points

/tmp/ipykernel_2390/725071667.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  schools_points['geometry']= schools.centroid


,visible,id,timestamp,changeset,tags,version,lon,lat,addr:city,addr:country,...,geometry,osm_type,addr:housename,ref,url,building:levels,internet_access,landuse,school,start_date
0,False,272517372,1567840265,0.0,None,3,8.239435,49.402039,Meckenheim,DE,...,POINT (8.23944 49.40204),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,False,295078565,1560223440,0.0,"{""wheelchair"":""no""}",5,8.249144,49.565739,None,None,...,POINT (8.24914 49.56574),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,316019351,1228123254,0.0,None,1,8.508860,49.508759,Mannheim,None,...,POINT (8.50886 49.50876),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,330955222,1595176296,0.0,"{""wheelchair"":""yes"",""wikidata"":""Q84995304""}",5,8.655658,49.471767,None,None,...,POINT (8.65566 49.47177),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,333298502,1488664212,0.0,"{""wheelchair"":""yes""}",6,8.656594,49.471947,None,None,...,POINT (8.65659 49.47195),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,NaN,19869100895,1615918892,0.0,"{""type"":""multipolygon"",""wikidata"":""Q2417212""}",1,NaN,NaN,None,None,...,POINT (8.42692 49.47811),relation,NaN,NaN,None,None,NaN,NaN,NaN,NaN
467,NaN,19881555370,1615918892,0.0,"{""type"":""multipolygon""}",1,NaN,NaN,None,None,...,POINT (8.42586 49.47761),relation,NaN,NaN,None,None,NaN,NaN,NaN,NaN
468,NaN,24682386538,1639739184,0.0,"{""short_name"":""KFG"",""type"":""multipolygon"",""wik...",1,NaN,NaN,None,None,...,POINT (8.69329 49.41086),relation,NaN,NaN,None,None,NaN,NaN,NaN,NaN
469,NaN,25833693929,1689361922,0.0,"{""operator:type"":""private"",""religion"":""christi...",2,NaN,NaN,Mannheim,None,...,POINT (8.46026 49.48605),relation,NaN,NaN,None,None,NaN,NaN,NaN,NaN


In [16]:
import h3pandas
hexgrid = counties.h3.polyfill_resample(10)

In [17]:
hexgrid_heidelberg = hexgrid.loc[hexgrid["name"] == 'Heidelberg']
hexgrid_heidelberg_centroids = hexgrid_heidelberg.copy()
hexgrid_heidelberg_centroids['geometry'] = hexgrid_heidelberg.centroid
hexgrid_heidelberg_centroids['id'] = range(53278047, 53278047 + len(hexgrid_heidelberg_centroids))
hexgrid_heidelberg_centroids

/tmp/ipykernel_2390/676297992.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hexgrid_heidelberg_centroids['geometry'] = hexgrid_heidelberg.centroid


,index,visible,boundary,id,timestamp,version,geometry,tags,osm_type,name,ref,website,admin_level,border_type,changeset
h3_polyfill,,,,,,,,,,,,,,,
8a1faad6c4affff,27,NaN,administrative,53278047,1695228268,32,POINT (8.71822 49.37129),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1fae6c9897fff,27,NaN,administrative,53278048,1695228268,32,POINT (8.66455 49.43098),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1fae6cb09ffff,27,NaN,administrative,53278049,1695228268,32,POINT (8.65556 49.40405),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faad63c27fff,27,NaN,administrative,53278050,1695228268,32,POINT (8.66811 49.35881),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faad64727fff,27,NaN,administrative,53278051,1695228268,32,POINT (8.64320 49.38446),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1fae6cdc97fff,27,NaN,administrative,53285857,1695228268,32,POINT (8.63696 49.44095),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faa99eb9ffff,27,NaN,administrative,53285858,1695228268,32,POINT (8.77933 49.40649),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faad61b5ffff,27,NaN,administrative,53285859,1695228268,32,POINT (8.70601 49.38360),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0


In [18]:
import datetime

In [20]:
layer = transport_network_rnv.transit_layer
layer.start_date

datetime.datetime(2023, 11, 24, 0, 0)

In [21]:
layer.end_date

datetime.datetime(2023, 12, 8, 23, 59, 59)

In [22]:
# stops_gdf['id'] = range(0, 0 + len(stops_gdf))

travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network_rnv,
    origins= hexgrid_heidelberg_centroids,
    destinations= schools_points,
    departure=datetime.datetime(2023, 12, 5, 6, 30),
    transport_modes=[
        r5py.TransportMode.WALK,
        r5py.TransportMode.TRANSIT
        ],
)

school_run = travel_time_matrix_computer.compute_travel_times()
school_run

,from_id,to_id,travel_time
0,53278047,272517372,NaN
1,53278047,295078565,NaN
2,53278047,316019351,NaN
3,53278047,330955222,79.0
4,53278047,333298502,78.0
...,...,...,...
3680860,53285861,19869100895,NaN
3680861,53285861,19881555370,NaN
3680862,53285861,24682386538,77.0
3680863,53285861,25833693929,NaN


In [23]:
non_na_traveltimes = school_run[~school_run['travel_time'].isna()]
non_na_traveltimes

,from_id,to_id,travel_time
3,53278047,330955222,79.0
4,53278047,333298502,78.0
5,53278047,333298506,80.0
8,53278047,622292949,54.0
13,53278047,736221550,55.0
...,...,...,...
3680840,53285861,1136310864,79.0
3680845,53285861,1154771353,112.0
3680852,53285861,4837893078,68.0
3680858,53285861,5854932161,79.0
